# Student Loan Risk with Deep Learning

In [1]:
# Install keras
!pip install keras

In [2]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [3]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [4]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [5]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

1    855
0    744
Name: credit_ranking, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [6]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [7]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [8]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [10]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
display(input_nodes)


11

In [11]:
# Define the number of hidden nodes for the first hidden layer
first_layer_neurons = 6

# Define the number of hidden nodes for the second hidden layer
second_layer_neurons = 3

# Define the number of neurons in the output layer
output_layer_neurons = 1

In [12]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=first_layer_neurons, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=second_layer_neurons, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(units=output_layer_neurons, activation="sigmoid"))

In [13]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 6)                 72        
                                                                 
 dense_1 (Dense)             (None, 3)                 21        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [14]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 1ms/step - loss: 0.7601 - accuracy: 0.4320
Epoch 2/50
38/38 [==============================] - 0s 1ms/step - loss: 0.7315 - accuracy: 0.4612
Epoch 3/50
38/38 [==============================] - 0s 1ms/step - loss: 0.7122 - accuracy: 0.4787
Epoch 4/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6979 - accuracy: 0.5179
Epoch 5/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6880 - accuracy: 0.5405
Epoch 6/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6799 - accuracy: 0.5588
Epoch 7/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6719 - accuracy: 0.5863
Epoch 8/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6637 - accuracy: 0.6130
Epoch 9/50
38/38 [==============================] - 0s 1ms/step - loss: 0.6556 - accuracy: 0.6347
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6460 - accuracy: 0.6489
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [16]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5126 - accuracy: 0.7450 - 178ms/epoch - 14ms/step
Loss: 0.5125683546066284, Accuracy: 0.7450000047683716


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [17]:
# Mount google colab drive to save keras model
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# Set the model's file path
file_path = Path("./drive/My Drive/student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [19]:
# Set the model's file path
file_path = Path("./drive/My Drive/student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [20]:
# Make predictions with the test data
student_loans_predictions = nn_imported.predict(X_test_scaled)

# Display a sample of the predictions
print(student_loans_predictions[:5])

13/13 [==============================] - 0s 1ms/step
[[0.54112417]
 [0.36279327]
 [0.8890224 ]
 [0.68380296]
 [0.979948  ]]


In [21]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(student_loans_predictions, columns=['predictions'])
#print(predictions_df)
predictions_df['predictions'] = predictions_df['predictions'].round()
print(predictions_df)

     predictions
0            1.0
1            0.0
2            1.0
3            1.0
4            1.0
..           ...
395          1.0
396          0.0
397          1.0
398          0.0
399          1.0

[400 rows x 1 columns]


### Step 4: Display a classification report with the y test data and predictions

In [22]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df['predictions']))

              precision    recall  f1-score   support

           0       0.70      0.79      0.74       188
           1       0.79      0.71      0.75       212

    accuracy                           0.74       400
   macro avg       0.75      0.75      0.74       400
weighted avg       0.75      0.74      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
>
> **Answer:**  
>>**a) College enrollment status - Target students that are currently enrolled, have been accepted to college, or have a college application pending.**  
**b) Student Loans Status - Does student already have student loans?  Students that already have student loans may be interested in getting another loan.**  
**c) Student Loan Interest - Is student interested learning about student loan options? Students that have an interest in student loan options may be more receptive to student loan recommendations.**  
>>**d) Student Financial Needs Assessment - This would help with recommending the appropriate student loans based on fnancial need.**  
>>**e) Student location/Residency - This will help determine eligibility for student loans**  
>>**f) Student grades/GPA - Good academic performance could be an indicator of a good student loan candidate**  
>

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
>
> **Answer:  Content-Based - This data is suitable for collecting from potential student loan applicants based on the loan product features and the student interests/preferences.**
>

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
>
> **Answer:**  
>> **Challenge 1:  Attribute assignment - There are many different types of student loans and they change frequently.  It will take a large amount of subject matter expertise and significant effort to maintain the loan attributes/tags in a loan recommentation system.**  
>> **Challenge 2:  Limited to Student Interests/Preferences - The model can only give suggestions based on the user's interests/preferences.  Students could miss out on student loan opportunities due to limited interests/preferences.**

